In [1]:
import pandas as pd
# import numpy as np
# import statsmodels.api as sm
# from statsmodels.stats.mediation import Mediation
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols


In [2]:
class Mediation:
    def __init__(self, data_file='data/cleaning data/participant_combined.csv'):
        self.data = pd.read_csv(data_file, delimiter=',')
        self.mediation_results = {}
        self.cvd_mediation_results = {}
        
    def preprocess_data(self):
        self.data = self.data.drop(columns=['eid', 'X34.0.0', 'Gender', 'Ethnicity', 'Migrant.Status', 'TDI.Tertiles', 'Highest.Qualification', 'House.Ownership', 'Income', 'Cohabiting', 'Living.Alone', 'AUDIT.Score', 'Smoker', 'Moderate.Physical.Activity', 'Longstanding.Illness', 'Diabetes', 'Cancer'])
        self.data.columns = self.data.columns.str.replace('.', '_', regex=False)
        self.data.columns = self.data.columns.str.replace('(', '', regex=False)
        self.data.columns = self.data.columns.str.replace(')', '', regex=False)
        self.data.columns = self.data.columns.str.replace('-', '', regex=False)
        self.data.columns = self.data.columns.str.replace('Remnant_Cholesterol_NonHDL,_NonLDL_Cholesterol', 
                                                          'Remnant_Cholesterol_NonHDL_NonLDL_Cholesterol')
        
        self.depression_atr = [
            'Depressed_At_Baseline', 'Loneliness', 'Social_Isolation', 'PHQ9_No_Info', 
            'PHQ9_Screen', 'PHQ9_Items', 'PHQ9_Severity', 'CIDI_MDD_No_Info', 
            'CIDI_MDD_Screen', 'CIDI_MDD_Response', 'CIDI_MDD_Severity', 'GAD_CIDI_Somatic'
        ]
        self.CVD = 'CVD'
        self.NMR_atr = [
            'Apolipoprotein_A1', 'Apolipoprotein_B',
            'Concentration_of_Chylomicrons_and_Extremely_Large_VLDL_Particles', 'Concentration_of_HDL_Particles', 'Concentration_of_IDL_Particles', 'Concentration_of_Large_HDL_Particles', 'Concentration_of_Large_LDL_Particles', 'Concentration_of_Large_VLDL_Particles', 'Concentration_of_LDL_Particles', 'Concentration_of_Medium_HDL_Particles', 'Concentration_of_Medium_LDL_Particles', 'Concentration_of_Medium_VLDL_Particles', 'Concentration_of_Small_HDL_Particles', 'Concentration_of_Small_LDL_Particles', 'Concentration_of_Small_VLDL_Particles', 'Concentration_of_Very_Large_HDL_Particles', 'Concentration_of_Very_Large_VLDL_Particles', 'Concentration_of_Very_Small_VLDL_Particles', 'Concentration_of_VLDL_Particles', 'Total_Concentration_of_Lipoprotein_Particles',
            'Average_Diameter_for_HDL_Particles', 'Average_Diameter_for_LDL_Particles', 'Average_Diameter_for_VLDL_Particles',
            'Cholesterol_in_Chylomicrons_and_Extremely_Large_VLDL', 'Cholesterol_in_IDL', 'Cholesterol_in_Large_HDL', 'Cholesterol_in_Large_LDL', 'Cholesterol_in_Large_VLDL', 'Cholesterol_in_Medium_HDL', 'Cholesterol_in_Medium_LDL', 'Cholesterol_in_Medium_VLDL', 'Cholesterol_in_Small_HDL', 'Cholesterol_in_Small_LDL', 'Cholesterol_in_Small_VLDL', 'Cholesterol_in_Very_Large_HDL', 'Cholesterol_in_Very_Large_VLDL', 'Cholesterol_in_Very_Small_VLDL', 'Clinical_LDL_Cholesterol', 'HDL_Cholesterol', 'LDL_Cholesterol', 'VLDL_Cholesterol', 'Remnant_Cholesterol_NonHDL_NonLDL_Cholesterol', 'Total_Cholesterol', 'Total_Cholesterol_Minus_HDLC',
            'Cholesteryl_Esters_in_Chylomicrons_and_Extremely_Large_VLDL', 'Cholesteryl_Esters_in_HDL', 'Cholesteryl_Esters_in_IDL', 'Cholesteryl_Esters_in_Large_HDL', 'Cholesteryl_Esters_in_Large_LDL', 'Cholesteryl_Esters_in_Large_VLDL', 'Cholesteryl_Esters_in_LDL', 'Cholesteryl_Esters_in_Medium_HDL', 'Cholesteryl_Esters_in_Medium_LDL', 'Cholesteryl_Esters_in_Medium_VLDL', 'Cholesteryl_Esters_in_Small_HDL', 'Cholesteryl_Esters_in_Small_LDL', 'Cholesteryl_Esters_in_Small_VLDL', 'Cholesteryl_Esters_in_Very_Large_HDL', 'Cholesteryl_Esters_in_Very_Large_VLDL', 'Cholesteryl_Esters_in_Very_Small_VLDL', 'Cholesteryl_Esters_in_VLDL', 'Total_Esterified_Cholesterol',
            'Free_Cholesterol_in_Chylomicrons_and_Extremely_Large_VLDL', 'Free_Cholesterol_in_HDL', 'Free_Cholesterol_in_IDL', 'Free_Cholesterol_in_Large_HDL', 'Free_Cholesterol_in_Large_LDL', 'Free_Cholesterol_in_Large_VLDL', 'Free_Cholesterol_in_LDL', 'Free_Cholesterol_in_Medium_HDL', 'Free_Cholesterol_in_Medium_LDL', 'Free_Cholesterol_in_Medium_VLDL', 'Free_Cholesterol_in_Small_HDL', 'Free_Cholesterol_in_Small_LDL', 'Free_Cholesterol_in_Small_VLDL', 'Free_Cholesterol_in_Very_Large_HDL', 'Free_Cholesterol_in_Very_Large_VLDL', 'Free_Cholesterol_in_Very_Small_VLDL', 'Free_Cholesterol_in_VLDL', 'Total_Free_Cholesterol',
            'Phospholipids_in_Chylomicrons_and_Extremely_Large_VLDL', 'Phospholipids_in_HDL', 'Phospholipids_in_IDL', 'Phospholipids_in_Large_HDL', 'Phospholipids_in_Large_LDL', 'Phospholipids_in_Large_VLDL', 'Phospholipids_in_LDL', 'Phospholipids_in_Medium_HDL', 'Phospholipids_in_Medium_LDL', 'Phospholipids_in_Medium_VLDL', 'Phospholipids_in_Small_HDL', 'Phospholipids_in_Small_LDL', 'Phospholipids_in_Small_VLDL', 'Phospholipids_in_Very_Large_HDL', 'Phospholipids_in_Very_Large_VLDL', 'Phospholipids_in_Very_Small_VLDL', 'Phospholipids_in_VLDL', 'Total_Phospholipids_in_Lipoprotein_Particles',
            'Triglycerides_in_Chylomicrons_and_Extremely_Large_VLDL', 'Triglycerides_in_HDL', 'Triglycerides_in_IDL', 'Triglycerides_in_Large_HDL', 'Triglycerides_in_Large_LDL', 'Triglycerides_in_Large_VLDL', 'Triglycerides_in_LDL', 'Triglycerides_in_Medium_HDL', 'Triglycerides_in_Medium_LDL', 'Triglycerides_in_Medium_VLDL', 'Triglycerides_in_Small_HDL', 'Triglycerides_in_Small_LDL', 'Triglycerides_in_Small_VLDL', 'Triglycerides_in_Very_Large_HDL', 'Triglycerides_in_Very_Large_VLDL', 'Triglycerides_in_Very_Small_VLDL', 'Triglycerides_in_VLDL', 'Total_Triglycerides',
            'Total_Lipids_in_Chylomicrons_and_Extremely_Large_VLDL', 'Total_Lipids_in_HDL', 'Total_Lipids_in_IDL', 'Total_Lipids_in_Large_HDL', 'Total_Lipids_in_Large_LDL', 'Total_Lipids_in_Large_VLDL', 'Total_Lipids_in_LDL', 'Total_Lipids_in_Lipoprotein_Particles', 'Total_Lipids_in_Medium_HDL', 'Total_Lipids_in_Medium_LDL', 'Total_Lipids_in_Medium_VLDL', 'Total_Lipids_in_Small_HDL', 'Total_Lipids_in_Small_LDL', 'Total_Lipids_in_Small_VLDL', 'Total_Lipids_in_Very_Large_HDL', 'Total_Lipids_in_Very_Large_VLDL', 'Total_Lipids_in_Very_Small_VLDL', 'Total_Lipids_in_VLDL',
            'Glycoprotein_Acetyls'
        ]

        relevant_columns = self.depression_atr + self.NMR_atr + [self.CVD]
        self.data = self.data[relevant_columns].dropna()
        scaler = StandardScaler()
        self.data[relevant_columns] = scaler.fit_transform(self.data[relevant_columns])

    def conduct_mediation_analysis(self, output_file='data/mediation results/mediation_results.csv'):
        for mediator in self.NMR_atr:
            try:
                # CVD predicts the mediator
                mediator_model_cvd = ols(f'{mediator} ~ {self.CVD}', data=self.data).fit()
                
                for dep_var in self.depression_atr:
                    # mediation analysis
                    model_1_cvd = ols(f'{dep_var} ~ {self.CVD}', data=self.data).fit()
                    direct_model_cvd = ols(f'{dep_var} ~ {self.CVD} + {mediator}', data=self.data).fit()

                    # extract parameters
                    a_cvd = mediator_model_cvd.params.get(self.CVD, None)
                    b_cvd = direct_model_cvd.params.get(mediator, None)
                    c_prime_cvd = direct_model_cvd.params.get(self.CVD, None)
                    c_cvd = model_1_cvd.params.get(self.CVD, None)

                    if mediator not in self.cvd_mediation_results:
                        self.cvd_mediation_results[mediator] = {}
                    self.cvd_mediation_results[mediator][dep_var] = {
                        'indirect_effect': a_cvd * b_cvd if a_cvd is not None and b_cvd is not None else None,
                        'direct_effect': c_prime_cvd,
                        'total_effect': c_cvd
                    }

                for dep_var in self.depression_atr:
                    # mediation analysis with depression variables
                    model_1 = ols(f'{dep_var} ~ {self.CVD}', data=self.data).fit()
                    direct_model = ols(f'{dep_var} ~ {self.CVD} + {mediator}', data=self.data).fit()

                    a = mediator_model_cvd.params.get(self.CVD, None)
                    b = direct_model.params.get(mediator, None)
                    c_prime = direct_model.params.get(self.CVD, None)
                    c = model_1.params.get(self.CVD, None)

                    if mediator not in self.mediation_results:
                        self.mediation_results[mediator] = {}
                    self.mediation_results[mediator][dep_var] = {
                        'indirect_effect': a * b if a is not None and b is not None else None,
                        'direct_effect': c_prime,
                        'total_effect': c
                    }

            except Exception as e:
                print(f"Error processing mediator {mediator}: {e}")
                self.mediation_results[mediator] = f"Error: {e}"
                self.cvd_mediation_results[mediator] = f"Error: {e}"

        # Combine results
        results_list = []
        for mediator, dep_results in self.mediation_results.items():
            for dep_var, result in dep_results.items():
                results_list.append([
                    mediator, 
                    dep_var, 
                    result.get('indirect_effect', None), 
                    result.get('direct_effect', None), 
                    result.get('total_effect', None)
                ])
        
        for mediator, cvd_results in self.cvd_mediation_results.items():
            for dep_var, result in cvd_results.items():
                results_list.append([
                    mediator, 
                    dep_var, 
                    result.get('indirect_effect', None), 
                    result.get('direct_effect', None), 
                    result.get('total_effect', None)
                ])
        
        combined_results_df = pd.DataFrame(results_list, columns=['Mediator', 'Attribute', 'Indirect_Effect', 'Direct_Effect', 'Total_Effect'])
        combined_results_df.to_csv(output_file, index=False)

        print(f"Combined mediation results saved to {output_file}")


In [3]:
if __name__ == "__main__":
    mediator_analysis = Mediation()
    mediator_analysis.preprocess_data()
    mediator_analysis.conduct_mediation_analysis()

# ~13 mins to run

Combined mediation results saved to data/mediation results/mediation_results.csv


Top Mediators

In [5]:
df = pd.read_csv('data/mediation results/mediation_results.csv')

grouped_df = df.groupby('Mediator').agg({
    'Indirect_Effect': 'mean',
    'Direct_Effect': 'mean',
    'Total_Effect': 'mean'
}).reset_index()

grouped_df.columns = ['Mediator', 'Average_Indirect_Effect', 'Average_Direct_Effect', 'Average_Total_Effect']
grouped_df.to_csv('data/mediation results/average_effects_by_mediator.csv', index=False)

top_10_df = grouped_df.nlargest(10, 'Average_Indirect_Effect')
top_10_df.to_csv('data/mediation results/top_10_mediators.csv', index=False)